# Рассмотрим работу Bag of Words

## Подготовка датасета

### Чтение текстов-примеров

Для начала импортируем нужные библиотеки


In [ ]:
import nltk 
nltk.download('punkt') 
import numpy as np
import random 
import string
import re 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Целью нашей нейронной сети будет предсказание, к какому именно стилю написания относится тот или иной отрывок текста в большей степени: научному, публицистическому или художественному. В качестве примера последнего стиля будет взят отрывок из романа Ф.М.Достоевского "Преступление и наказание" размером 5196 слов.

In [ ]:
Text = open("Художественный текст.txt")
artistic_text = Text.read()
Text.close()

print(len(artistic_text.split()))
artistic_text

5196


'В начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С — м переулке, на улицу и медленно, как бы в нерешимости, отправился к К — ну мосту.\nОн благополучно избегнул встречи с своею хозяйкой на лестнице. Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру. Квартирная же хозяйка его, у которой он нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже, в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу. И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение, которого стыдился и от которого морщился. Он был должен кругом хозяйке и боялся с нею встретиться.\nНе то чтоб он был так труслив и забит, совсем даже напротив; но с некоторого времени он был в раздражительном и напряженном состоянии, п

В качестве примера научного текста был взят сборник из следующих научных статей:

Зацепин М.Н. ПРЕПОДАВАНИЕ БАЗ ДАННЫХ ДЛЯ СТУДЕНТОВ НЕТЕХНИЧЕСКИХ СПЕЦИАЛЬНОСТЕЙ // Вопросы технических и физико-математических наук в свете современных исследований: сб. ст. по матер. XL междунар. науч.-практ. конф. № 6(32). – Новосибирск: СибАК, 2021. – С. 22-27.

Такуадина А.И. ЧИСЛЕННЫЕ МЕТОДЫ РЕШЕНИЯ ЗАДАЧ ФАРМАКОКИНЕТИКИ // Вопросы технических и физико-математических наук в свете современных исследований: сб. ст. по матер. III-IV междунар. науч.-практ. конф. № 3-4(3). – Новосибирск: СибАК, 2018. – С. 53-57.

Верещагина Е.А., Третьяк А.В. ИССЛЕДОВАНИЕ СОВРЕМЕННЫХ ЯЗЫКОВ ПРОГРАММИРОВАНИЯ // Вопросы технических и физико-математических наук в свете современных исследований: сб. ст. по матер. XXIII-XXIV междунар. науч.-практ. конф. № 1-2(19). – Новосибирск: СибАК, 2020. – С. 39-43.

Елисеев Н.А., Федоров С.А., Антонов О.Д. ОБЗОР УГРОЗ БЕЗОПАСНОСТИ WEB-ПРИЛОЖЕНИЙ // Вопросы технических наук в свете современных исследований: сб. ст. по матер. V-VI междунар. науч.-практ. конф. № 1(4). – Новосибирск: СибАК, 2018. – С. 18-23.

Юрьев Д.Р., Рогова О.С. СРАВНИТЕЛЬНЫЙ АНАЛИЗ ДВУХФАКТОРНОЙ АУТЕНТИФИКАЦИИ // Технические науки - от теории к практике: сб. ст. по матер. LXXI междунар. науч.-практ. конф. № 6(66). – Новосибирск: СибАК, 2017. – С. 46-51.

In [ ]:
Text = open("Научный текст.txt")
science_text = Text.read()
Text.close()

print(len(science_text.split()))
science_text

4851


'Для большинства специалистов, получающих высшее образование по программе бакалавриата и т.д. знакомство с базами данных, как с изнанкой хранения данных в цифровых системах, чрезвычайно полезно. Знакомство с базами данных и СУБД позволяет расширить общеобразовательный кругозор, развить логику мышления, улучшить абстрагирование при мышлении, что является необходимостью процесса высшего образования. Помимо этого, пользователи информационных систем (коими является абсолютное большинство) начинают понимать принципы и механизмы, лежащие в основе работы с данными различных программ или интерфейсов, принципы работы фильтрации данных при выводе.\n\nНачалом должно быть знакомство с файловыми системами и разметкой носителей. Данные темы редко рассматриваются в школьных курсах в достаточной мере (чаще вообще не рассматриваются). Знакомство включает краткую историю, назначение, развитие, современное состояние, структуру и правила адресаций. Сравнительно рассматривается история развития баз данных,

И, наконец, запишем публицистический текст - в качестве примера была взята сборка из статей с Habr.com

In [ ]:
Text = open("Публицистический текст.txt")
journalistic_text = Text.read()
Text.close()

print(len(journalistic_text.split()))
journalistic_text

4934


'1.1 Введение\n\nБлагодаря машинному обучению программист не обязан писать инструкции, учитывающие все возможные проблемы и содержащие все решения. Вместо этого в компьютер (или отдельную программу) закладывают алгоритм самостоятельного нахождения решений путём комплексного использования статистических данных, из которых выводятся закономерности и на основе которых делаются прогнозы.\n\nТехнология машинного обучения на основе анализа данных берёт начало в 1950 году, когда начали разрабатывать первые программы для игры в шашки. За прошедшие десятилетий общий принцип не изменился. Зато благодаря взрывному росту вычислительных мощностей компьютеров многократно усложнились закономерности и прогнозы, создаваемые ими, и расширился круг проблем и задач, решаемых с использованием машинного обучения.\n\nЧтобы запустить процесс машинного обучение, для начала необходимо загрузить в компьютер Датасет(некоторое количество исходных данных), на которых алгоритм будет учиться обрабатывать запросы. Нап

### Подготовка данных


Разобьем каждый текст на предложения - для каждого выведем первые 10, чтобы проверить, что все верно

In [ ]:
science_corpus = nltk.sent_tokenize(science_text)
science_corpus[0:10]

['Для большинства специалистов, получающих высшее образование по программе бакалавриата и т.д.',
 'знакомство с базами данных, как с изнанкой хранения данных в цифровых системах, чрезвычайно полезно.',
 'Знакомство с базами данных и СУБД позволяет расширить общеобразовательный кругозор, развить логику мышления, улучшить абстрагирование при мышлении, что является необходимостью процесса высшего образования.',
 'Помимо этого, пользователи информационных систем (коими является абсолютное большинство) начинают понимать принципы и механизмы, лежащие в основе работы с данными различных программ или интерфейсов, принципы работы фильтрации данных при выводе.',
 'Началом должно быть знакомство с файловыми системами и разметкой носителей.',
 'Данные темы редко рассматриваются в школьных курсах в достаточной мере (чаще вообще не рассматриваются).',
 'Знакомство включает краткую историю, назначение, развитие, современное состояние, структуру и правила адресаций.',
 'Сравнительно рассматривается ис

In [ ]:
journalistic_corpus = nltk.sent_tokenize(journalistic_text)
journalistic_corpus[0:10]

['1.1 Введение\n\nБлагодаря машинному обучению программист не обязан писать инструкции, учитывающие все возможные проблемы и содержащие все решения.',
 'Вместо этого в компьютер (или отдельную программу) закладывают алгоритм самостоятельного нахождения решений путём комплексного использования статистических данных, из которых выводятся закономерности и на основе которых делаются прогнозы.',
 'Технология машинного обучения на основе анализа данных берёт начало в 1950 году, когда начали разрабатывать первые программы для игры в шашки.',
 'За прошедшие десятилетий общий принцип не изменился.',
 'Зато благодаря взрывному росту вычислительных мощностей компьютеров многократно усложнились закономерности и прогнозы, создаваемые ими, и расширился круг проблем и задач, решаемых с использованием машинного обучения.',
 'Чтобы запустить процесс машинного обучение, для начала необходимо загрузить в компьютер Датасет(некоторое количество исходных данных), на которых алгоритм будет учиться обрабатыва

In [ ]:
artistic_corpus = nltk.sent_tokenize(artistic_text)
artistic_corpus[0:10]

['В начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С — м переулке, на улицу и медленно, как бы в нерешимости, отправился к К — ну мосту.',
 'Он благополучно избегнул встречи с своею хозяйкой на лестнице.',
 'Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру.',
 'Квартирная же хозяйка его, у которой он нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже, в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу.',
 'И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение, которого стыдился и от которого морщился.',
 'Он был должен кругом хозяйке и боялся с нею встретиться.',
 'Не то чтоб он был так труслив и забит, совсем даже напротив; но с некоторого времени он был в раздражительном и н

Приведем предложения в нормализованный вид - переведем все буквы в нижний регистр и удалим все знаки пунктуации, а также приведем все слова в начальную форму.

In [ ]:
# !pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

for i in range(len(science_corpus)):
    science_corpus[i] = science_corpus[i].lower()
    science_corpus[i] = re.sub(r'\W',' ',science_corpus[i])
    science_corpus[i] = re.sub(r'\s+',' ',science_corpus[i])
    ex_sentence = ""
    for word in science_corpus[i].split():
        p = morph.parse(word)[0] 
        ex_sentence += p.normal_form + " "
    science_corpus[i] = ex_sentence

science_corpus[0:10]    

['для большинство специалист получать высокий образование по программа бакалавриат и так далее ',
 'знакомство с база данные как с изнанка хранение данные в цифровой система чрезвычайно полезно ',
 'знакомство с база данные и субд позволять расширить общеобразовательный кругозор развить логика мышление улучшить абстрагирование при мышление что являться необходимость процесс высокий образование ',
 'помимо это пользователь информационный система кой являться абсолютный большинство начинать понимать принцип и механизм лежать в основа работа с данные различный программа или интерфейс принцип работа фильтрация данные при вывод ',
 'начало должный быть знакомство с файловый система и разметка носитель ',
 'дать тема редко рассматриваться в школьный курс в достаточный мера частый вообще не рассматриваться ',
 'знакомство включать краткий история назначение развитие современный состояние структура и правило адресация ',
 'сравнительно рассматриваться история развитие база данные и сопоставлят

In [ ]:
for i in range(len(journalistic_corpus)):
    journalistic_corpus[i] = journalistic_corpus[i].lower()
    journalistic_corpus[i] = re.sub(r'\W',' ',journalistic_corpus[i])
    journalistic_corpus[i] = re.sub(r'\s+',' ',journalistic_corpus[i])
    ex_sentence = ""
    for word in journalistic_corpus[i].split():
        p = morph.parse(word)[0] 
        ex_sentence += p.normal_form + " "
    journalistic_corpus[i] = ex_sentence
journalistic_corpus[0:10]  

['1 1 введение благодаря машинный обучение программист не обязать писать инструкция учитывать всё возможный проблема и содержать всё решение ',
 'вместо это в компьютер или отдельный программа закладывать алгоритм самостоятельный нахождение решение путём комплексный использование статистический данные из который выводиться закономерность и на основа который делаться прогноз ',
 'технология машинный обучение на основа анализ данные брать начало в 1950 год когда начать разрабатывать первый программа для игра в шашка ',
 'за прошедшее десятилетие общий принцип не измениться ',
 'зато благодаря взрывной рост вычислительный мощность компьютер многократно усложниться закономерность и прогноз создавать они и расшириться круг проблема и задача решать с использование машинный обучение ',
 'чтобы запустить процесс машинный обучение для начало необходимо загрузить в компьютер датасет некоторый количество исходный данные на который алгоритм быть учиться обрабатывать запрос ',
 'например мочь быть 

In [ ]:
for i in range(len(artistic_corpus)):
    artistic_corpus[i] = artistic_corpus[i].lower()
    artistic_corpus[i] = re.sub(r'\W',' ',artistic_corpus[i])
    artistic_corpus[i] = re.sub(r'\s+',' ',artistic_corpus[i])
    ex_sentence = ""
    for word in artistic_corpus[i].split():
        p = morph.parse(word)[0] 
        ex_sentence += p.normal_form + " "
    artistic_corpus[i] = ex_sentence
artistic_corpus[0:10]  

['в начало июль в чрезвычайно жаркое время под вечер один молодой человек выйти из свой каморка который нанимать от жилец в с м переулок на улица и медленно как бы в нерешимость отправиться к к ну мост ',
 'он благополучно избегнуть встреча с свой хозяйка на лестница ',
 'каморка он приходиться под самый кровля высокий пятиэтажный дом и походить более на шкаф чем на квартира ',
 'квартирный же хозяйка он у который он нанимать этот каморка с обед и прислуга помещаться один лестница ниже в отдельный квартира и каждый раз при выход на улица он непременно надо быть проходить мимо хозяйкин кухня почти всегда настежь отворить на лестница ',
 'и каждый раз молодой человек проходить мимо чувствовать какой то болезненный и трусливый ощущение который стыдиться и от который морщиться ',
 'он быть должный кругом хозяйка и бояться с она встретиться ',
 'не то чтоб он быть так трусливый и забитый совсем даже напротив но с некоторый время он быть в раздражительный и напрячь состояние похожий на ипохо

Соберем датасет: пусть научному тексту соответствует цифра 1, публицистике - 2, художественной литературе - 3

In [ ]:
import pandas as pd

science_y = [1] * len(science_corpus)
journalistic_y = [2] * len(journalistic_corpus)
artistic_y = [3] * len(artistic_corpus)

science_data = {'Text' : science_corpus, 'Num' : science_y}
journalistic_data = {'Text' : journalistic_corpus, 'Num' : journalistic_y}
artistic_data = {'Text' : artistic_corpus, 'Num' : artistic_y}

df_s = pd.DataFrame(science_data)
df_j = pd.DataFrame(journalistic_data)
df_a = pd.DataFrame(artistic_data)

df = pd.concat([df_s, df_j, df_a])
df

,Text,Num
0,для большинство специалист получать высокий об...,1
1,знакомство с база данные как с изнанка хранени...,1
2,знакомство с база данные и субд позволять расш...,1
3,помимо это пользователь информационный система...,1
4,начало должный быть знакомство с файловый сист...,1
...,...,...
317,да и то статский советник клопшток иван иванов...,3
318,не только деньга за шитьё полдюжины голландски...,3
319,а тут ребятишки голодный а тут катерина иванов...,3
320,лежать я тогда ну да уж что,3


## Создание модели машинного обучения

Выпишем все слова, которые встречаются в текстах выше, и определим количество их появлений, тем самым классифицировав по частоте

In [ ]:
wordfreq = {}
for sentence in df["Text"]:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1
print(len(wordfreq.keys()))
wordfreq["стол"]

3665


4

Уберем самые частые слова (по большей части это предлоги и союзы, которые есть во всех трех текстах, а значит, не сильно влияют на точность предсказания), а также самые редкие слова (которые, по сути, так же слабо будут влиять на предсказания модели из-за своей редкости)

Всего у нас 3665 различных слов - возьмем интервал из самых частых от 500 до 2500 слов - всего 2000 слов

In [ ]:
import heapq
most_freq = heapq.nlargest(4000, wordfreq, key=wordfreq.get)
most_freq = most_freq[500:1500]
most_freq[0:5]

['вычисление', 'заключаться', 'привести', 'поиск', 'квадрат']

Применим алгоритм Bag of Words - токенизируем предложения так, чтобы получились вектора, показывающие, есть ли такое слово в предложении или нет, а затем, транспонировав матрицу, получим вектора для каждого слова, которые показывают, в каких предложениях оно есть.

In [ ]:
sentence_vectors = []
for sentence in df['Text']:
    sentence_tokens = nltk.word_tokenize(sentence)
    sent_vec = []
    for token in most_freq:
        if token in sentence_tokens:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)
sentence_vectors = np.asarray(sentence_vectors)
word_vectors = sentence_vectors.T

Создадим итоговый датасет, в котором привяжем совокупность слов в предложении к типу стиля

In [ ]:
from pandas.core.indexes.base import default_index
i : int = 0
for word in word_vectors:
    df[most_freq[i]] = word
    i+=1
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


,Text,Num,вычисление,заключаться,привести,поиск,квадрат,играть,про,сделать,...,чтоб,раздражительный,положение,перестать,останавливаться,проскользнуть,видать,поразить,пустяк,странный
0,для большинство специалист получать высокий об...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,знакомство с база данные как с изнанка хранени...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,знакомство с база данные и субд позволять расш...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,помимо это пользователь информационный система...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,начало должный быть знакомство с файловый сист...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,да и то статский советник клопшток иван иванов...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
318,не только деньга за шитьё полдюжины голландски...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
319,а тут ребятишки голодный а тут катерина иванов...,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
320,лежать я тогда ну да уж что,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Проверка различных классификаторов

Таким образом, получаем итоговый датасет. Обучим его с помощью некоторых моделей классификации из библиотеки Scikit-Learn

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X = df.drop(columns=['Text', 'Num'])
y = df['Num']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = KNeighborsClassifier(n_neighbors = 3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.3602941176470588


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = AdaBoostClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.4117647058823529


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.49264705882352944


In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6617647058823529


In [ ]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.7352941176470589


## Проверка на практике

In [ ]:
N = int(input("Введите количество текстов: "))
for i in range(N):
    test_text = input("Введите текст, стиль которого нужно определить: ")

    test_text = test_text.lower()
    test_text = re.sub(r'\W',' ',test_text)
    test_text = re.sub(r'\s+',' ',test_text)
    ex_sentence = ""
    for word in test_text.split():
        p = morph.parse(word)[0] 
        ex_sentence += p.normal_form + " "
    test_text = ex_sentence
    test_tokens = nltk.word_tokenize(ex_sentence)
    test_vec = []
    for token in most_freq:
        if token in test_tokens:
            test_vec.append(1)
        else:
            test_vec.append(0)
    test_vec = np.array(test_vec)
    test = {}
    i : int = 0
    for word in test_vec:
        test[most_freq[i]] = word
        i+=1
    test = pd.DataFrame(test, index=[0])

    test_mark = model.predict(test)

    if (test_mark == 1):
        print("Скорее всего, это научный текст")
    if (test_mark == 2):
        print("Я думаю, этот текст ближе к публицистике")
    if (test_mark == 3):
        print("Я бы сказала, что это художественный стиль")

Введите количество текстов: 3
Введите текст, стиль которого нужно определить: Всем привет, сегодня я расскажу вам о своих взлетах и падениях при разработке нейронных сетей
Я думаю, этот текст ближе к публицистике
Введите текст, стиль которого нужно определить: Сравнение некоторых алгоритмов машинного обучения позволяет получить интересные результаты
Скорее всего, это научный текст
Введите текст, стиль которого нужно определить: Вновь наступила осень, и листья, гонимые ветром, все падали и падали на мокрую от утреннего дождя дорогу
Я бы сказала, что это художественный стиль


# Рассмотрим работу TF-IDF

Во многом подготовка датасета совпадает с решением для первой модели - отличие только в цифрах - теперь в датасете будут не нули и единицы, а, скажем так, степени "важности" слов

## Подготовка данных

### Чтение текстов-примеров

Получение текстов документов полностью совпадает с предыдущим примером.

Для начала импортируем нужные библиотеки


In [ ]:
import nltk 
nltk.download('punkt') 
import numpy as np
import random 
import string
import re 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Целью нашей нейронной сети будет предсказание, к какому именно стилю написания относится тот или иной отрывок текста в большей степени: научному, публицистическому или художественному. В качестве примера последнего стиля будет взят отрывок из романа Ф.М.Достоевского "Преступление и наказание" размером 5196 слов.

In [ ]:
Text = open("Художественный текст.txt")
artistic_text = Text.read()
Text.close()

print(len(artistic_text.split()))
artistic_text

5196


'В начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С — м переулке, на улицу и медленно, как бы в нерешимости, отправился к К — ну мосту.\nОн благополучно избегнул встречи с своею хозяйкой на лестнице. Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру. Квартирная же хозяйка его, у которой он нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже, в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу. И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение, которого стыдился и от которого морщился. Он был должен кругом хозяйке и боялся с нею встретиться.\nНе то чтоб он был так труслив и забит, совсем даже напротив; но с некоторого времени он был в раздражительном и напряженном состоянии, п

В качестве примера научного текста был взят сборник из следующих научных статей:

Зацепин М.Н. ПРЕПОДАВАНИЕ БАЗ ДАННЫХ ДЛЯ СТУДЕНТОВ НЕТЕХНИЧЕСКИХ СПЕЦИАЛЬНОСТЕЙ // Вопросы технических и физико-математических наук в свете современных исследований: сб. ст. по матер. XL междунар. науч.-практ. конф. № 6(32). – Новосибирск: СибАК, 2021. – С. 22-27.

Такуадина А.И. ЧИСЛЕННЫЕ МЕТОДЫ РЕШЕНИЯ ЗАДАЧ ФАРМАКОКИНЕТИКИ // Вопросы технических и физико-математических наук в свете современных исследований: сб. ст. по матер. III-IV междунар. науч.-практ. конф. № 3-4(3). – Новосибирск: СибАК, 2018. – С. 53-57.

Верещагина Е.А., Третьяк А.В. ИССЛЕДОВАНИЕ СОВРЕМЕННЫХ ЯЗЫКОВ ПРОГРАММИРОВАНИЯ // Вопросы технических и физико-математических наук в свете современных исследований: сб. ст. по матер. XXIII-XXIV междунар. науч.-практ. конф. № 1-2(19). – Новосибирск: СибАК, 2020. – С. 39-43.

Елисеев Н.А., Федоров С.А., Антонов О.Д. ОБЗОР УГРОЗ БЕЗОПАСНОСТИ WEB-ПРИЛОЖЕНИЙ // Вопросы технических наук в свете современных исследований: сб. ст. по матер. V-VI междунар. науч.-практ. конф. № 1(4). – Новосибирск: СибАК, 2018. – С. 18-23.

Юрьев Д.Р., Рогова О.С. СРАВНИТЕЛЬНЫЙ АНАЛИЗ ДВУХФАКТОРНОЙ АУТЕНТИФИКАЦИИ // Технические науки - от теории к практике: сб. ст. по матер. LXXI междунар. науч.-практ. конф. № 6(66). – Новосибирск: СибАК, 2017. – С. 46-51.

In [ ]:
Text = open("Научный текст.txt")
science_text = Text.read()
Text.close()

print(len(science_text.split()))
science_text

4851


'Для большинства специалистов, получающих высшее образование по программе бакалавриата и т.д. знакомство с базами данных, как с изнанкой хранения данных в цифровых системах, чрезвычайно полезно. Знакомство с базами данных и СУБД позволяет расширить общеобразовательный кругозор, развить логику мышления, улучшить абстрагирование при мышлении, что является необходимостью процесса высшего образования. Помимо этого, пользователи информационных систем (коими является абсолютное большинство) начинают понимать принципы и механизмы, лежащие в основе работы с данными различных программ или интерфейсов, принципы работы фильтрации данных при выводе.\n\nНачалом должно быть знакомство с файловыми системами и разметкой носителей. Данные темы редко рассматриваются в школьных курсах в достаточной мере (чаще вообще не рассматриваются). Знакомство включает краткую историю, назначение, развитие, современное состояние, структуру и правила адресаций. Сравнительно рассматривается история развития баз данных,

И, наконец, запишем публицистический текст - в качестве примера была взята сборка из статей с Habr.com

In [ ]:
Text = open("Публицистический текст.txt")
journalistic_text = Text.read()
Text.close()

print(len(journalistic_text.split()))
journalistic_text

4934


'1.1 Введение\n\nБлагодаря машинному обучению программист не обязан писать инструкции, учитывающие все возможные проблемы и содержащие все решения. Вместо этого в компьютер (или отдельную программу) закладывают алгоритм самостоятельного нахождения решений путём комплексного использования статистических данных, из которых выводятся закономерности и на основе которых делаются прогнозы.\n\nТехнология машинного обучения на основе анализа данных берёт начало в 1950 году, когда начали разрабатывать первые программы для игры в шашки. За прошедшие десятилетий общий принцип не изменился. Зато благодаря взрывному росту вычислительных мощностей компьютеров многократно усложнились закономерности и прогнозы, создаваемые ими, и расширился круг проблем и задач, решаемых с использованием машинного обучения.\n\nЧтобы запустить процесс машинного обучение, для начала необходимо загрузить в компьютер Датасет(некоторое количество исходных данных), на которых алгоритм будет учиться обрабатывать запросы. Нап

### Подготовка данных


Обработка предложений так же мало чем отличается от Bag of Words. Основное отличие в том, как будет создаваться основа для датасета (следующий пункт)

Разобьем каждый текст на предложения - для каждого выведем первые 10, чтобы проверить, что все верно

In [ ]:
science_corpus = nltk.sent_tokenize(science_text)
science_corpus[0:10]

['Для большинства специалистов, получающих высшее образование по программе бакалавриата и т.д.',
 'знакомство с базами данных, как с изнанкой хранения данных в цифровых системах, чрезвычайно полезно.',
 'Знакомство с базами данных и СУБД позволяет расширить общеобразовательный кругозор, развить логику мышления, улучшить абстрагирование при мышлении, что является необходимостью процесса высшего образования.',
 'Помимо этого, пользователи информационных систем (коими является абсолютное большинство) начинают понимать принципы и механизмы, лежащие в основе работы с данными различных программ или интерфейсов, принципы работы фильтрации данных при выводе.',
 'Началом должно быть знакомство с файловыми системами и разметкой носителей.',
 'Данные темы редко рассматриваются в школьных курсах в достаточной мере (чаще вообще не рассматриваются).',
 'Знакомство включает краткую историю, назначение, развитие, современное состояние, структуру и правила адресаций.',
 'Сравнительно рассматривается ис

In [ ]:
journalistic_corpus = nltk.sent_tokenize(journalistic_text)
journalistic_corpus[0:10]

['1.1 Введение\n\nБлагодаря машинному обучению программист не обязан писать инструкции, учитывающие все возможные проблемы и содержащие все решения.',
 'Вместо этого в компьютер (или отдельную программу) закладывают алгоритм самостоятельного нахождения решений путём комплексного использования статистических данных, из которых выводятся закономерности и на основе которых делаются прогнозы.',
 'Технология машинного обучения на основе анализа данных берёт начало в 1950 году, когда начали разрабатывать первые программы для игры в шашки.',
 'За прошедшие десятилетий общий принцип не изменился.',
 'Зато благодаря взрывному росту вычислительных мощностей компьютеров многократно усложнились закономерности и прогнозы, создаваемые ими, и расширился круг проблем и задач, решаемых с использованием машинного обучения.',
 'Чтобы запустить процесс машинного обучение, для начала необходимо загрузить в компьютер Датасет(некоторое количество исходных данных), на которых алгоритм будет учиться обрабатыва

In [ ]:
artistic_corpus = nltk.sent_tokenize(artistic_text)
artistic_corpus[0:10]

['В начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в С — м переулке, на улицу и медленно, как бы в нерешимости, отправился к К — ну мосту.',
 'Он благополучно избегнул встречи с своею хозяйкой на лестнице.',
 'Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более на шкаф, чем на квартиру.',
 'Квартирная же хозяйка его, у которой он нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже, в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу.',
 'И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение, которого стыдился и от которого морщился.',
 'Он был должен кругом хозяйке и боялся с нею встретиться.',
 'Не то чтоб он был так труслив и забит, совсем даже напротив; но с некоторого времени он был в раздражительном и н

Приведем предложения в нормализованный вид - переведем все буквы в нижний регистр и удалим все знаки пунктуации, а также приведем все слова в начальную форму.

In [ ]:
# !pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

for i in range(len(science_corpus)):
    science_corpus[i] = science_corpus[i].lower()
    science_corpus[i] = re.sub(r'\W',' ',science_corpus[i])
    science_corpus[i] = re.sub(r'\s+',' ',science_corpus[i])
    ex_sentence = ""
    for word in science_corpus[i].split():
        p = morph.parse(word)[0] 
        ex_sentence += p.normal_form + " "
    science_corpus[i] = ex_sentence

science_corpus[0:10]    

['для большинство специалист получать высокий образование по программа бакалавриат и так далее ',
 'знакомство с база данные как с изнанка хранение данные в цифровой система чрезвычайно полезно ',
 'знакомство с база данные и субд позволять расширить общеобразовательный кругозор развить логика мышление улучшить абстрагирование при мышление что являться необходимость процесс высокий образование ',
 'помимо это пользователь информационный система кой являться абсолютный большинство начинать понимать принцип и механизм лежать в основа работа с данные различный программа или интерфейс принцип работа фильтрация данные при вывод ',
 'начало должный быть знакомство с файловый система и разметка носитель ',
 'дать тема редко рассматриваться в школьный курс в достаточный мера частый вообще не рассматриваться ',
 'знакомство включать краткий история назначение развитие современный состояние структура и правило адресация ',
 'сравнительно рассматриваться история развитие база данные и сопоставлят

In [ ]:
for i in range(len(journalistic_corpus)):
    journalistic_corpus[i] = journalistic_corpus[i].lower()
    journalistic_corpus[i] = re.sub(r'\W',' ',journalistic_corpus[i])
    journalistic_corpus[i] = re.sub(r'\s+',' ',journalistic_corpus[i])
    ex_sentence = ""
    for word in journalistic_corpus[i].split():
        p = morph.parse(word)[0] 
        ex_sentence += p.normal_form + " "
    journalistic_corpus[i] = ex_sentence
journalistic_corpus[0:10]  

['1 1 введение благодаря машинный обучение программист не обязать писать инструкция учитывать всё возможный проблема и содержать всё решение ',
 'вместо это в компьютер или отдельный программа закладывать алгоритм самостоятельный нахождение решение путём комплексный использование статистический данные из который выводиться закономерность и на основа который делаться прогноз ',
 'технология машинный обучение на основа анализ данные брать начало в 1950 год когда начать разрабатывать первый программа для игра в шашка ',
 'за прошедшее десятилетие общий принцип не измениться ',
 'зато благодаря взрывной рост вычислительный мощность компьютер многократно усложниться закономерность и прогноз создавать они и расшириться круг проблема и задача решать с использование машинный обучение ',
 'чтобы запустить процесс машинный обучение для начало необходимо загрузить в компьютер датасет некоторый количество исходный данные на который алгоритм быть учиться обрабатывать запрос ',
 'например мочь быть 

In [ ]:
for i in range(len(artistic_corpus)):
    artistic_corpus[i] = artistic_corpus[i].lower()
    artistic_corpus[i] = re.sub(r'\W',' ',artistic_corpus[i])
    artistic_corpus[i] = re.sub(r'\s+',' ',artistic_corpus[i])
    ex_sentence = ""
    for word in artistic_corpus[i].split():
        p = morph.parse(word)[0] 
        ex_sentence += p.normal_form + " "
    artistic_corpus[i] = ex_sentence
artistic_corpus[0:10]  

['в начало июль в чрезвычайно жаркое время под вечер один молодой человек выйти из свой каморка который нанимать от жилец в с м переулок на улица и медленно как бы в нерешимость отправиться к к ну мост ',
 'он благополучно избегнуть встреча с свой хозяйка на лестница ',
 'каморка он приходиться под самый кровля высокий пятиэтажный дом и походить более на шкаф чем на квартира ',
 'квартирный же хозяйка он у который он нанимать этот каморка с обед и прислуга помещаться один лестница ниже в отдельный квартира и каждый раз при выход на улица он непременно надо быть проходить мимо хозяйкин кухня почти всегда настежь отворить на лестница ',
 'и каждый раз молодой человек проходить мимо чувствовать какой то болезненный и трусливый ощущение который стыдиться и от который морщиться ',
 'он быть должный кругом хозяйка и бояться с она встретиться ',
 'не то чтоб он быть так трусливый и забитый совсем даже напротив но с некоторый время он быть в раздражительный и напрячь состояние похожий на ипохо

Соберем датасет: пусть научному тексту соответствует цифра 1, публицистике - 2, художественной литературе - 3

In [ ]:
import pandas as pd

science_y = [1] * len(science_corpus)
journalistic_y = [2] * len(journalistic_corpus)
artistic_y = [3] * len(artistic_corpus)

science_data = {'Text' : science_corpus, 'Num' : science_y}
journalistic_data = {'Text' : journalistic_corpus, 'Num' : journalistic_y}
artistic_data = {'Text' : artistic_corpus, 'Num' : artistic_y}

df_s = pd.DataFrame(science_data)
df_j = pd.DataFrame(journalistic_data)
df_a = pd.DataFrame(artistic_data)

df = pd.concat([df_s, df_j, df_a])
df

,Text,Num
0,для большинство специалист получать высокий об...,1
1,знакомство с база данные как с изнанка хранени...,1
2,знакомство с база данные и субд позволять расш...,1
3,помимо это пользователь информационный система...,1
4,начало должный быть знакомство с файловый сист...,1
...,...,...
317,да и то статский советник клопшток иван иванов...,3
318,не только деньга за шитьё полдюжины голландски...,3
319,а тут ребятишки голодный а тут катерина иванов...,3
320,лежать я тогда ну да уж что,3


## Создание итогового датасета

In [ ]:
wordfreq = {}
for sentence in df["Text"]:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1
print(len(wordfreq.keys()))
wordfreq["стол"]

3665


4

In [ ]:
word_idf_values = {}
for token in most_freq:
    doc_containing_word = 0
    for document in df["Text"]:
        if token in nltk.word_tokenize(document):
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(df["Text"])/(1 + doc_containing_word))

In [ ]:
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for document in df["Text"]:
        doc_freq = 0
        for word in nltk.word_tokenize(document):
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(nltk.word_tokenize(document))
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector

In [ ]:
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

tf_idf_model = np.asarray(tfidf_values)
tf_idf_model

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from pandas.core.indexes.base import default_index
i : int = 0
for word in tf_idf_model:
    df[most_freq[i]] = word
    i+=1
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


,Text,Num,вычисление,заключаться,привести,поиск,квадрат,играть,про,сделать,...,чтоб,раздражительный,положение,перестать,останавливаться,проскользнуть,видать,поразить,пустяк,странный
0,для большинство специалист получать высокий об...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,знакомство с база данные как с изнанка хранени...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,знакомство с база данные и субд позволять расш...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,помимо это пользователь информационный система...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,начало должный быть знакомство с файловый сист...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,да и то статский советник клопшток иван иванов...,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
318,не только деньга за шитьё полдюжины голландски...,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
319,а тут ребятишки голодный а тут катерина иванов...,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
320,лежать я тогда ну да уж что,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Проверка различных классификаторов

Таким образом, получаем итоговый датасет. Обучим его с помощью тех же моделей, что и в Bag of Words

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X = df.drop(columns=['Text', 'Num'])
y = df['Num']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = KNeighborsClassifier(n_neighbors = 3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.3713235294117647


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = AdaBoostClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.41544117647058826


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.5110294117647058


In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6102941176470589


In [ ]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42, shuffle = True)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6580882352941176


## Проверка на практике

In [ ]:
N = int(input("Введите количество текстов: "))
for i in range(N):
    test_text = input("Введите текст, стиль которого нужно определить: ")

    test_text = test_text.lower()
    test_text = re.sub(r'\W',' ',test_text)
    test_text = re.sub(r'\s+',' ',test_text)
    ex_sentence = ""
    for word in test_text.split():
        p = morph.parse(word)[0] 
        ex_sentence += p.normal_form + " "
    test_text = ex_sentence
    test_tokens = nltk.word_tokenize(ex_sentence)
    test_vec = []
    for token in most_freq:
        if token in test_tokens:
            test_vec.append(1)
        else:
            test_vec.append(0)
    test_vec = np.array(test_vec)
    test = {}
    i : int = 0
    for word in test_vec:
        test[most_freq[i]] = word
        i+=1
    test = pd.DataFrame(test, index=[0])

    test_mark = model.predict(test)

    if (test_mark == 1):
        print("Скорее всего, это научный текст")
    if (test_mark == 2):
        print("Я думаю, этот текст ближе к публицистике")
    if (test_mark == 3):
        print("Я бы сказала, что это художественный стиль")

Введите количество текстов: 3
Введите текст, стиль которого нужно определить: Всем привет, сегодня я расскажу вам о своих взлетах и падениях при разработке нейронных сетей
Я думаю, этот текст ближе к публицистике
Введите текст, стиль которого нужно определить: Сравнение некоторых алгоритмов машинного обучения позволяет получить интересные результаты
Скорее всего, это научный текст
Введите текст, стиль которого нужно определить: Вновь наступила осень, и листья, гонимые ветром, все падали и падали на мокрую от утреннего дождя дорогу
Я бы сказала, что это художественный стиль
